# Road Traffic Accidents in Switzerland

In this project, we work with the data from Switzerland's road accidents.

In [1]:
import helpers
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import requests

%load_ext autoreload
%autoreload 2

## Scraping the data

Writing to the "Office Fédéral des Statistiques" was unsuccessful, so we resort to web scraping.

The scraped data comes from http://map.donneesaccidents.ch/ (or https://map.geo.admin.ch/?topic=vu&lang=fr&bgLayer=ch.swisstopo.pixelkarte-grau&layers=ch.astra.unfaelle-personenschaeden_alle&layers_timestamp=&catalogNodes=1318, after redirections).

#### Layers

The map contains the following layers:
* Accidents with personal injury
* Accidents with fatalities
* Accidents involving a pedestrian
* Accidents involving a bicycle
* Accidents involving a motorcycle

There is no information regarding accidents where nobody is unjured, since by law only accidents in which someone is injured require police intervention. This explains why only these are archived.

#### Getting detailed accident data

The map only displays markers for each accident. When one or more markers are selected, a web query retrieves the information related to the selected markers. Using postman, we analysed such queries and found a way to select several or all markers on the map.

This is achieved by defining a rectangle, and querying for all markers inside it. We declare a rectangle that encompasses all Switzerland. This behaviour can be simulated on the browser by holding the ctrl key (or cmd on mac) and dragging a rectangle over the map.
The returned data is paginated, so it is necessary to go over all the pages to obtain all detailed accident data.

Example query parameters:
```
geometry:443999.04209536605,39001.6733318335,870499.0420953662,303001.67333183356
geometryFormat:geojson
geometryType:esriGeometryEnvelope
imageDisplay:1536,759,96
lang:en
layers:all:LAYER_PARAM
offset:200
mapExtent:269999.04209536605,9501.673331833561,1037999.042095366,389001.67333183356
returnGeometry:true
tolerance:5
```

`LAYER_PARAM` can be replaced by the appropriate layer name and `offset` is optional.

In [12]:
# Import raw data from http://map.donneesaccidents.ch/
data = helpers.import_data()

Scraping layer ch.astra.unfaelle-personenschaeden_alle
Layer scraped : 90600 records
Scraping layer ch.astra.unfaelle-personenschaeden_getoetete
Layer scraped : 1343 records
Scraping layer ch.astra.unfaelle-personenschaeden_fussgaenger
Layer scraped : 11738 records
Scraping layer ch.astra.unfaelle-personenschaeden_fahrraeder
Layer scraped : 18104 records
Scraping layer ch.astra.unfaelle-personenschaeden_motorraeder
Layer scraped : 19676 records
Scraped data: 141461 records


In [18]:
pickle.dump(data, open('data/data_raw.pickle', 'wb'))

In [30]:
raw_df = pd.DataFrame(data)
raw_df.sample(5)

,bbox,featureId,geometry,geometryType,id,layerBodId,layerName,properties,type
40540,"[711582.0, 237485.0, 711582.0, 237485.0]",2288AB5886B801E0E0530A83942732C3,"{'coordinates': [[711582.0, 237485.0]], 'type'...",Feature,2288AB5886B801E0E0530A83942732C3,ch.astra.unfaelle-personenschaeden_alle,Accidents avec dommages corporels,{'severitycategory_fr': 'accident avec blessés...,Feature
89667,"[650379.0, 211879.0, 650379.0, 211879.0]",FDC33805335300FCE0430A8394272D3F,"{'coordinates': [[650379.0, 211879.0]], 'type'...",Feature,FDC33805335300FCE0430A8394272D3F,ch.astra.unfaelle-personenschaeden_alle,Accidents avec dommages corporels,{'severitycategory_fr': 'accident avec blessés...,Feature
16808,"[786378.0, 191612.0, 786378.0, 191612.0]",A7397F33ADD40190E0430A8394270190,"{'coordinates': [[786378.0, 191612.0]], 'type'...",Feature,A7397F33ADD40190E0430A8394270190,ch.astra.unfaelle-personenschaeden_alle,Accidents avec dommages corporels,{'severitycategory_fr': 'accident avec blessés...,Feature
17136,"[646844.0, 227879.0, 646844.0, 227879.0]",0170898A08D80126E0530A83942700AE,"{'coordinates': [[646844.0, 227879.0]], 'type'...",Feature,0170898A08D80126E0530A83942700AE,ch.astra.unfaelle-personenschaeden_alle,Accidents avec dommages corporels,{'severitycategory_fr': 'accident avec blessés...,Feature
38940,"[567319.0, 122230.0, 567319.0, 122230.0]",AF53FCC8B4E001A0E0430A83942701A0,"{'coordinates': [[567319.0, 122230.0]], 'type'...",Feature,AF53FCC8B4E001A0E0430A83942701A0,ch.astra.unfaelle-personenschaeden_alle,Accidents avec dommages corporels,{'severitycategory_fr': 'accident avec blessés...,Feature


## Exploring and cleaning the data

In [2]:
data = pickle.load(open('data/data_raw.pickle', 'rb'))

In this initial data dump, we see the following fields:
* **`bbox`** Redundant coordinates to place the accident on the map.
* **`featureId`** Unique ID for an accident.
* **`geometry`**  GeoJSON *geometry* object for the accident. Example:
```
{'coordinates': [[663430.0, 193370.0]], 'type': 'MultiPoint'}
```
* **`geometryType`** GeoJSON type
* **`id`** Redundancy of `featureId`
* **`layerBodId`** Map layers the accident belongs to
* **`layerName`** Layer name
* **`properties`** Accident properties. Example:
```
{'accidentday_de': 'Mittwoch / 12h-13h / Mai 2014',
 'accidentday_fr': 'mercredi / 12h-13h / mai 2014',
 'accidentday_it': 'Mercoledì / 12h-13h / Maggio 2014',
 'accidenttype_de': 'Abbiegeunfall',
 'accidenttype_fr': 'accident en quittant une route',
 'accidenttype_it': 'Incidente nello svoltare',
 'accidenttypecode': 3,
 'accidentyear': 2014,
 'canton': 'BE',
 'fsocommunecode': '0979',
 'label': 'Abbiegeunfall',
 'roadtype_de': 'Nebenstrasse',
 'roadtype_fr': 'route secondaire',
 'roadtype_it': 'Strada secondaria',
 'roadtypecode': 433,
 'severitycategory_de': 'Unfall mit Leichtverletzten',
 'severitycategory_fr': 'accident avec blessés légers',
 'severitycategory_it': 'Incidente con feriti leggeri',
 'severitycategorycode': 'ULV'}
```
* **`type`** Redundancy of `geometryType`

The first step to be able to fully manipulate this data is to flatten it:

In [3]:
flat_data = [helpers.flatten(entry) for entry in data]

In [45]:
accidents_df = pd.DataFrame(flat_data)
accidents_df.columns

Index(['bbox', 'featureId', 'geometryType', 'geometry_coordinates',
       'geometry_type', 'id', 'layerBodId', 'layerName',
       'properties_accidentday_de', 'properties_accidentday_fr',
       'properties_accidentday_it', 'properties_accidenttype_de',
       'properties_accidenttype_fr', 'properties_accidenttype_it',
       'properties_accidenttypecode', 'properties_accidentyear',
       'properties_canton', 'properties_fsocommunecode', 'properties_label',
       'properties_roadtype_de', 'properties_roadtype_fr',
       'properties_roadtype_it', 'properties_roadtypecode',
       'properties_severitycategory_de', 'properties_severitycategory_fr',
       'properties_severitycategory_it', 'properties_severitycategorycode',
       'type'],
      dtype='object')

To clean the data, we start by removing the columns that are irrelevant or redundant, such as:
* **`bbox`** coordinates for the accident are available in the `geometry_coordinates` column.
* **`featureId`** we keep `id`
* **`geometryType` and `type`** Innecessary GeoJSON info
* **`layerBodId`** we keep `layerName`
* **Translations**: all translated fields are only kept in french (english not available).

After removing all 

In [46]:
accidents_df = accidents_df.drop([
    'bbox',
    'featureId',
    'geometryType',
    'type',
    'layerName',
    'properties_accidentday_de',
    'properties_accidentday_it',
    'properties_severitycategory_de',
    'properties_severitycategory_it',
    'properties_severitycategory_fr',
    'properties_roadtype_de',
    'properties_roadtype_it',
    'properties_accidenttype_de',
    'properties_accidenttype_it',
    'properties_accidenttype_fr',
    'properties_roadtype_fr',
    'properties_label',
    'geometry_type'
], axis=1)

# Select one set of coordinates for the accident
accidents_df.geometry_coordinates = accidents_df.geometry_coordinates.apply(lambda x: x[0])

accidents_df = accidents_df.set_index('id')
accidents_df = accidents_df.sort_index()

In [47]:
accidents_df.head(5)

,geometry_coordinates,layerBodId,properties_accidentday_fr,properties_accidenttypecode,properties_accidentyear,properties_canton,properties_fsocommunecode,properties_roadtypecode,properties_severitycategorycode
id,,,,,,,,,
000632B7E4500082E0530A83942743EA,"[647210.0, 177439.0]",ch.astra.unfaelle-personenschaeden_fahrraeder,samedi / 22h-23h / juillet 2014,4,2014,BE,0573,432,USV
000632B7E4500082E0530A83942743EA,"[647210.0, 177439.0]",ch.astra.unfaelle-personenschaeden_alle,samedi / 22h-23h / juillet 2014,4,2014,BE,0573,432,USV
000750DA26760038E0530A8394272B98,"[685704.0, 252693.0]",ch.astra.unfaelle-personenschaeden_alle,jeudi / 16h-17h / juillet 2014,2,2014,ZH,0069,430,ULV
0007F58D137500F0E0530A839427CD36,"[750913.0, 256375.0]",ch.astra.unfaelle-personenschaeden_motorraeder,mardi / 06h-07h / juin 2014,0,2014,SG,3212,432,USV
0007F58D151500F0E0530A839427CD36,"[689567.0, 231957.0]",ch.astra.unfaelle-personenschaeden_alle,mercredi / 08h-09h / juin 2014,0,2014,ZH,0133,430,USV


In [48]:
# Accident layers
accidents_df['withBodilyDamage'] = accidents_df.layerBodId == 'ch.astra.unfaelle-personenschaeden_alle'
accidents_df['involvesBicycles'] = accidents_df.layerBodId == 'ch.astra.unfaelle-personenschaeden_fahrraeder'
accidents_df['involvesMotorcycles'] = accidents_df.layerBodId == 'ch.astra.unfaelle-personenschaeden_motorraeder'
accidents_df['involvesPedestrians'] = accidents_df.layerBodId == 'ch.astra.unfaelle-personenschaeden_fussgaenger'
accidents_df['withFatality'] = accidents_df.layerBodId == 'ch.astra.unfaelle-personenschaeden_getoetete'
accidents_df = accidents_df.drop('layerBodId', axis=1)

# Time properties
weekdays_fr_en = {
    'lundi': 'Monday',
    'mardi': 'Tuesday',
    'mercredi': 'Wednesday',
    'jeudi': 'Thursday',
    'vendredi': 'Friday',
    'samedi': 'Saturday',
    'dimanche': 'Sunday'
}
accidents_df.properties_accidentday_fr = accidents_df.properties_accidentday_fr.apply(lambda x: str.split(x, '/'))
accidents_df['weekday'] = accidents_df.properties_accidentday_fr.apply(lambda x: weekdays_fr_en[x[0].strip(' ')])
accidents_df['time'] = accidents_df.properties_accidentday_fr.apply(lambda x: x[1].strip(' '))
accidents_df['month'] = accidents_df.properties_accidentday_fr.apply(lambda x: x[2].split()[0])
accidents_df['year'] = accidents_df.properties_accidentyear
accidents_df = accidents_df.drop('properties_accidentday_fr', axis=1)
accidents_df = accidents_df.drop('properties_accidentyear', axis=1)

# Accident severity
severity_codes = {
    'ULV': 'light injuries',
    'USV': 'heavy injuries',
    'UGT': 'deceased'
}
accidents_df.properties_severitycategorycode = accidents_df.properties_severitycategorycode.apply(lambda x: severity_codes[x])

# Accident type
accident_type = {
    0: 'Skidding or control loss',
    1: 'Overtaking or changing roads',
    2: 'Collision',
    3: 'Leaving a road',
    4: "Joining a road",
    5: 'Crossing a road',
    6: 'Frontal collision',
    7: 'Parking accident',
    8: 'With pedestrians',
    9: 'With animals',
    99: 'Other'
}
accidents_df.properties_accidenttypecode = accidents_df.properties_accidenttypecode.apply(lambda x: accident_type[x])

accidents_df.sample(5)

,geometry_coordinates,properties_accidenttypecode,properties_canton,properties_fsocommunecode,properties_roadtypecode,properties_severitycategorycode,withBodilyDamage,involvesBicycles,involvesMotorcycles,involvesPedestrians,withFatality,weekday,time,month,year
id,,,,,,,,,,,,,,,
BECCBAF1DB8E01EEE0430A8394271953,"[718005.0, 197356.0]",With animals,GL,1631,432,heavy injuries,False,False,True,False,False,Friday,21h-22h,mai,2012
1EC41F940515000AE0530A8394277BAC,"[678845.0, 259513.0]",Joining a road,ZH,0090,433,light injuries,True,False,False,False,False,Wednesday,15h-16h,juillet,2015
B46F8176652F01FCE0430A83942701FC,"[599547.0, 200804.0]",Crossing a road,BE,0351,433,light injuries,True,False,False,False,False,Saturday,18h-19h,octobre,2011
0216EE83DF7600AAE0530A8394274D06,"[574688.0, 196369.0]",Crossing a road,FR,2275,432,light injuries,True,False,False,False,False,Sunday,10h-11h,juillet,2014
198CA2C2124E0196E0530A8394278689,"[500703.0, 115516.0]",Collision,GE,6608,432,light injuries,True,False,False,False,False,Friday,12h-13h,juillet,2015


In [49]:
accidents_df = accidents_df.groupby(by=accidents_df.index).agg({
    'geometry_coordinates': 'first',
    'properties_accidenttypecode': 'first',
    'properties_canton': 'first',
    'properties_fsocommunecode': 'first',
    'properties_roadtypecode': 'first',
    'properties_severitycategorycode': 'first',
    'withBodilyDamage': np.max,
    'involvesBicycles': np.max,
    'involvesMotorcycles': np.max,
    'involvesPedestrians': np.max,
    'withFatality': np.max,
    'weekday': 'first',
    'time': 'first',
    'month': 'first',
    'year': 'first'
})

print('There are', len(accidents_df), 'accidents')

There are 75022 accidents


In [50]:
accidents_df.to_csv('data/accidents.csv')

# Data analysis

1) Accidents par rapport au temps<br>
2) Corrélation nombre/type d'accident avec les endroits (Valais ivresse)<br>
3) Tracker des anomalies (fin/début d'une série d'accident) et essayer d'en trouver la cause<br>

In [9]:
plot_feature_combination(df, ['accident_year', 'month'])

In [10]:
plot_feature_combination(df, ['canton','day'])

In [11]:
plot_feature_combination(df, ['road_type','day'])

In [12]:
plot_feature_combination(df, ['time', 'severity_category'])

In [13]:
len(df.fsocommunecode.unique())


2194

In [14]:
plot_feature_combination(df, ['accident_type_code', 'severity_category_code'])

In [15]:
plot_all_feature_combinations(df, ['coordinates'], 2)

***Generating feature combinations***
-> Done
***Plotting feature combinations***
-> Done
